In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map['length']
objective2 = map['crossing']
objective3 = map['obstacle_free_width']

objectives = ('length', 'crossing')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map, approach='primal')
nodes = G.nodes
# print(len(nodes))
edges = G.edges
# print(len(max(nx.connected_components(G), key=len)))

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [4]:
from shapely.geometry import Point
# start_node = gpd.GeoDataFrame({'geometry': [Point(122245.37633330293,
#                                                   486126.8581684635)]})
# end_node = gpd.GeoDataFrame({'geometry': [Point(122247.04588395767,
#                                                   486167.91526857053)]})
# fig, ax = plt.subplots(figsize=(14,14), dpi=600)
# # All nodes and edges
# nx.draw(G, {n:[n[0], n[1]] for n in list(G.nodes)}, ax=ax, node_size=3)
# # Start & end node
# start_node.plot(ax=ax, color='red')
# end_node.plot(ax=ax, color='purple')
# plt.show()

In [5]:
# print(nodes)

In [6]:
#Pick random ones or pick manually that make sense - to experiment
#Smaller map:
S = (122245.37633330293, 486126.8581684635)
T = (122246.77932030056, 486223.5791244763)
#Small map:
# S = (120558.58272730978, 486088.5913559315)
# T = (120683.06067979027, 485777.31229122455)

In [7]:
# Initialise the Gaussian process for 2 objectives
gp = gaussian_process.GPPairwise(num_objectives=2, std_noise=0.01, kernel_width=0.15,prior_mean_type='zero', seed=None)

In [8]:
P = [] #Pareto set
p = [] #paths computed by Dijkstra's algorithm
val_vector_p = [] #value vectors w.r.t. p, i.e., v^{p_1}, v^{p_2}

# Path initialisation
for i in objectives:
    p = nx.shortest_path(G, source=S, target=T, weight=i, method='dijkstra') #Dijkstra's algorithm
    P.append(p)

    val_obj1 = nx.path_weight(G, path=p, weight='length') #Returns total cost associated with the path and weight. In other words, it returns the value of the path.
    val_obj2 = nx.path_weight(G, path=p, weight='crossing')
    val_vector_p.append(np.array([val_obj1, val_obj2]))

In [9]:
val_vector_p

[array([165.21,   2.  ]), array([227.59,   2.  ])]

In [10]:
C = [min(val_vector_p[0][0], val_vector_p[1][0]), min(val_vector_p[0][1], val_vector_p[1][1])] #Candidate Targets, i.e., the most optimistic points
C = [np.array(C)]
C

[array([165.21,   2.  ])]

In [11]:
# User ranking: Compare paths in P
user_preference = utils_user.UserPreference(num_objectives=2, std_noise=0.1)

In [12]:
add_noise = True
ground_utility = user_preference.get_preference(val_vector_p, add_noise=add_noise) #This is the ground-truth utility, i.e., the true utility
print(ground_utility)

[1404214.83672325 1404214.65871866]


/Users/alisatodorova/Desktop/Thesis/Interactive-MOPPA/lmzintgraf_gp_pref_elicit/gp_utilities/utils_user.py:254: RuntimeWarning: overflow encountered in exp
  y += 1. / (1 + np.exp(- x * (a-i) + (b+i)))


In [13]:
# Add the comparisons to GP
comparisons = dataset.DatasetPairwise(num_objectives=2)

comparisons.add_single_comparison(val_vector_p[np.argmax(ground_utility)], val_vector_p[np.argmin(ground_utility)]) #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

[[165.21   2.  ]
 [227.59   2.  ]]


In [14]:
# Find the path the user likes best and has the maximum a posteriori (MAP) estimate
u_v, _ = gp.get_predictive_params(val_vector_p, True) #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
print(u_v)

[ 0.02560512 -0.02560512]


In [15]:
p_star_index = np.argmax(u_v)
p_star_index

0

In [16]:
p_star = P[p_star_index]
p_star

[(122245.37633330293, 486126.8581684635),
 (122254.86602688645, 486129.80052856216),
 (122264.35426393585, 486132.74411788705),
 (122273.84081337851, 486135.69101398275),
 (122283.19823912054, 486138.9678026403),
 (122284.1816391946, 486139.19731384714),
 (122290.12705889999, 486148.4950394649),
 (122296.3898333376, 486151.2069021054),
 (122293.35529070727, 486160.70100324217),
 (122290.30564961619, 486170.1963395323),
 (122289.24584929895, 486173.25169782597),
 (122284.2947638598, 486179.51092348614),
 (122286.01308887315, 486183.8814735307),
 (122282.69556966702, 486193.2948693236),
 (122279.36739295325, 486202.70485648955),
 (122276.034562429, 486212.11332275654),
 (122274.00277185065, 486217.2508821529),
 (122271.878266905, 486221.2167929504),
 (122262.38569380266, 486218.13414513733),
 (122254.26477155345, 486215.7695028797),
 (122253.09793657108, 486219.18429932056),
 (122246.77932030056, 486223.5791244763)]

In [17]:
input_domain = np.array(C) #set of candidate targets
print(input_domain)

[[165.21   2.  ]]


In [18]:
# Initialise the acquisition function
acq_fun = acquisition_function.DiscreteAcquirer(input_domain=input_domain, query_type='ranking', seed=123, acquisition_type='expected improvement')

In [19]:
# TODO: The next code cells are in a while-loop
# while C:
#     expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
#     t_index = np.argmax(expected_improvement)
#     t = C[t_index]
#     C.remove(t)
#
# # t_index
# print(t)

In [20]:
expected_improvement = acquisition_function.get_expected_improvement(input_domain, gp, acq_fun.history)
print(expected_improvement)

[0.14549021]


In [21]:
t_index = np.argmax(expected_improvement)
t_index

0

In [22]:
t = input_domain[t_index]
t

array([165.21,   2.  ])

In [23]:
# Remove t from C
# C.remove(t)
# C

In [57]:
# # Check if IDs of the nodes = IDs of the edges
# for n in G:
#     for e in G.edges(data=True):
#         if n == e[0]:
#             print("True")
#         elif n == e[1]:
#             print("Next")
#         else:
#             print("Error")


In [97]:
import copy
v_n = {}
threshold=1e-8
max_iter=1000
obj = 'length'
next_node = {}

for n in G:
    v_n[n] = np.inf
    if n == T:
        v_n[n] = 0

for i in range(max_iter): #or until convergence
    v_n_copy = copy.deepcopy(v_n)

    converged = True

    # print("iteration:", i)
    for e in G.edges(data=True):
        n1, n2 = e[0], e[1]
        # if v_n[n1] != np.inf or v_n[n2] != np.inf:
        #     print(e[0], e[1], ":", v_n[n1], v_n[n2])

        # print(n2 in G)
        cost = e[2][obj]
            # print(cost)

            # print(e)
        result1 = min(cost + v_n[n2], v_n[n1])
        result2 = min(cost + v_n[n1], v_n[n2])
            # print("result1:", result1, "sum1:", cost + v_n[n2])
            # print("result2", result2, "sum2:", cost + v_n[n1])
        if v_n[n1] != result1 or v_n[n2] != result2:
            converged = False

        if v_n[n1] != result1:
            next_node[n1] = n2

        if v_n[n2] != result2:
            next_node[n2] = n1

        v_n[n1] = result1
        v_n[n2] = result2

        # if n1 == T or n2==T:
        #     # print(result1, result2)
        #     print(v_n[n1], v_n[n2])


    # converged = all(n in v_n_copy and abs(v_n[n] - v_n_copy[n]) < threshold for n in v_n)
    if converged:
        break



        # if v_n[n1] != np.inf:
        #     print(n1,v_n[n1])
        # if v_n[n2] != np.inf:
        #     print(n2,v_n[n2])

print(v_n)
print(next_node)
# import single_vi_iter
# lower_bounds = single_vi_iter.single_value_iter(G)
lower_bounds = v_n

# lower_bounds = dict(sorted(lower_bounds.items(), key=lambda x: lower_bounds[x[0]],
#                        reverse=True))
#
# lower_bounds

# S = (122245.37633330293, 486126.8581684635)
# T = (122246.77932030056, 486223.5791244763)

{(122245.37633330293, 486126.8581684635): 165.21, (122254.86602688645, 486129.80052856216): 155.22, (122264.35426393585, 486132.74411788705): 145.23, (122273.84081337851, 486135.69101398275): 135.23999999999998, (122283.19823912054, 486138.9678026403): 125.24999999999999, (122284.1816391946, 486139.19731384714): 124.23999999999998, (122287.28805009159, 486137.97386694513): 127.58999999999997, (122291.40829399273, 486140.643590483): 132.52999999999997, (122289.53424561612, 486128.31132216856): 137.57999999999998, (122297.7854183222, 486139.47700177913): 139.09999999999997, (122301.24255780602, 486130.1701925009): 149.08999999999997, (122304.47424118801, 486120.76208520425): 159.07999999999998, (122256.54763350038, 486170.91398380254): 104.02, (122247.04588395767, 486167.91526857053): 114.00999999999999, (122237.50744934028, 486164.9669819529): 123.99999999999999, (122227.969152229, 486162.0189639581): 133.98999999999998, (122218.43104837791, 486159.0709819753): 143.98, (122289.245849298

In [96]:
#TODO: Fix this
i = 0  # track iterations of the algorithm
max_iter=1000

stack = [(S, 0, [S])]  # (starting node, cost, path), where cost is from previous_state to S, path is from S to current_state (i.e., S)

obj = 'length'

while stack:
    current_node, cost, path = stack.pop()  # cost=total cost up to the current_node

    if current_node == T:
        print(f"Path {path} with cost {cost}")
    #
    # for e in G.edges(data=True):
    #     n1, n2 = e[0], e[1]
    #     cost = e[2][obj]
    #     total_cost = cost + next_node_cost
    for next_node in G[current_node]:
        for key in G:
            total_cost = key + next_node
            new_path = path + [next_node] # Add the neighbor to the path
        # print(new_path)
        stack.append((next_node, total_cost, new_path))
        # print(stack)
        stack.sort(key=lambda x: lower_bounds[x[0]],
                       reverse=True)  # Sorts in descending order w.r.t. the lower bound

    i += 1
    if max_iter is not None and i >= max_iter:
        print(path)
        print("The algorithm has reached the given maximum iterations, but has found no solution.")
        break


print("The algorithm has terminated, but no solution was found.")

Path [(122245.37633330293, 486126.8581684635), (122254.86602688645, 486129.80052856216), (122264.35426393585, 486132.74411788705), (122273.84081337851, 486135.69101398275), (122283.19823912054, 486138.9678026403), (122284.1816391946, 486139.19731384714), (122290.12705889999, 486148.4950394649), (122296.3898333376, 486151.2069021054), (122293.35529070727, 486160.70100324217), (122290.30564961619, 486170.1963395323), (122289.24584929895, 486173.25169782597), (122284.2947638598, 486179.51092348614), (122286.01308887315, 486183.8814735307), (122282.69556966702, 486193.2948693236), (122279.36739295325, 486202.70485648955), (122276.034562429, 486212.11332275654), (122274.00277185065, 486217.2508821529), (122271.878266905, 486221.2167929504), (122262.38569380266, 486218.13414513733), (122254.26477155345, 486215.7695028797), (122253.09793657108, 486219.18429932056), (122246.77932030056, 486223.5791244763)] with cost (122320.31466476223, 486327.5294561802, 122246.77932030056, 486223.5791244763)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Inner-loop approach
import dfs_lower
p_s = dfs_lower.dfs_lower(G, S, t, lower_bounds)
print(p_s)

In [ ]:
#TODO: Line 15 of pseudocode is unclear to me how it should be in code.
# If v^p improves in the target region
# because you've identified a new value vector on the PCS. If you stop once the utility no longer improves, I think this can result in stopping prematurely. Specifically, imagine you have a current partial Pareto front of (10,0) and (0,10) the user model u((10,0)) is the current best. The target vector is (10,10)  and when you run DFS, you get one of the possible vectors in the target region. You get (1,9) out of the the call to DFS, and the u((1,9)) < u((10,0)) even after querying the user about it. Now here, you shouldn't stop, because the true best - (7,3) for example, is still possible to find.
#you're just not going to improve on that with a newly found vector
#So improving on the acquisition function by identifying a new point is impossible as
# 1) you were searching at an optimistic estimate (target), so the actually found value will be worse than the target
# 2) finding new points, and querying the user reduces uncertainty

In [ ]:
P = P.append(p_s)

In [ ]:
#TODO: Can't test because of p_s (see above)
#Compare p^s to p^∗ and add comparison to the GP ▷User ranking, i.e., is the new path preferred to the current, maximum one?

val_vector_new_paths = [] #value vectors of paths p^s and p^∗

for i in objectives:
    val_p_s = nx.path_weight(G, path=p_s, weight=i)
    val_p_star = nx.path_weight(G, path=p_star, weight=i)

    val_vector_new_paths.append(np.array([val_p_s, val_p_star]))


In [ ]:
ranking_new_paths = user_preference.get_preference(val_vector_new_paths, add_noise=add_noise)
print(ranking_new_paths)

In [ ]:
# Add the comparisons to GP
comparisons.add_single_comparison(val_vector_new_paths[np.argmax(ranking_new_paths)], val_vector_new_paths[
    np.argmin(ranking_new_paths)])  #This way we are performing user ranking of their preferences
print(comparisons.datapoints)
gp.update(comparisons)

In [ ]:
u_v_p_s, _ = gp.get_predictive_params(val_p_s, True)  #The maximum a posteriori (MAP) estimate is the mean from gaussian_process.get_predictive_params()
u_v_p_star, _ = gp.get_predictive_params(val_p_star, True)

In [ ]:
# if u(v^{p^s}) > u(v^{p^*}) then
if u_v_p_s > u_v_p_star:
    #p^∗ ← p^s
    p_star = p_s
# end if

In [ ]:
# Compute new candidate targets based on v^{p^s} and add to C
C = [min(val_p_s[0][0], val_p_s[1][0]), min(val_p_s[0][1], val_p_s[1][1])]
C

In [ ]:
# end if
# end while
# return p^∗, v^{p^*}
